In [ ]:

fieldnames = [
    'CaseNumber',
    'Filed',
    'Closed',
    'Judge',
    'Plantiff',
    'PlantiffAttorney',
    'Defendant',
    'DefendantAttorney',
    'Issue',
    'FiledBy',
    'Disposed',
    'source'
]


def parties_and_their_attorneys(case):

    result = []

    plaintiff = ''
    plaintiff_attorney = ''
    defendant = ''
    defendant_attorney = ''

    for party in case.parties:

        if party['type'] == 'Defendant':
            defendant = party['name']
            result.append(defendant)

        if party['type'] == 'Plaintiff':
            plaintiff = party['name']
            result.append(plaintiff)

        if (plaintiff != '') and (defendant != ''):
            break

    for attorney in case.attorneys:
        if attorney['representing'] == defendant:
            defendant_attorney = attorney['name']

        if attorney['representing'] == plaintiff:
            plaintiff_attorney = attorney['name']

        if (plaintiff_attorney != '') and (defendant_attorney != ''):
            break

    return [plaintiff, plaintiff_attorney, defendant, defendant_attorney]


def case_row(case):
    row_data = []

    # 'CaseNumber'
    row_data.append(case.index)

    # 'Filed'
    row_data.append(case.filed)

    # 'Closed'
    row_data.append(case.closed)

    # 'Judge'
    row_data.append(case.judge)

    # 'Plantiff','Defendant', 'PlantiffAttorney','DefendantAttorney'

    row_data += parties_and_their_attorneys(case)

    if case.issues:
        first_issue = case.issues[0]

    # 'Issue'
        row_data.append(first_issue['Issue'])

    # 'FiledBy'
        row_data.append(first_issue['Filed By'])

    # 'Disposed',
        disposed = ''

        if first_issue['parties']:
            disposed = first_issue['parties'][0]['disposed']
        row_data.append(disposed)
    else:
        row_data += ['', '', '']


    row_data.append(case.source)

    return row_data


In [25]:
import oscn
import csv

eviction_params = {
    'dcct': 26, 
    'county': 'tulsa',
    'filed_after': '06/01/2020',
    'filed_before': '06/11/2020'
}

small_claims = oscn.find.CaseIndexes(**eviction_params)

def case_generator(find_code, cases):
    for case_index in small_claims:
        case = oscn.request.Case(case_index)
#       exclude if VERIFICATION OF COMPLIANCE OF CARES ACT also filed
        CARES_COMPL = 'VERIFICATION OF COMPLIANCE OF CARES ACT'
        if CARES_COMPL in case.docket.text:
            break
        if find_code in case.text:
            has_code = lambda min: min['code'] == find_code
            minutes_with_code = filter(has_code, case.docket)
            if len(list(minutes_with_code)) > 0:
                print(case.source)
                yield case
                
eviction_cases = case_generator('SCFED1', small_claims)



In [26]:
import numpy as np
import pandas as pd

columns = ['Filed','County','Type','Judge','Case Number', 'Source']
case_data = lambda c: [c.filed, c.county, c.type, c.judge,  c.index, c.source]
%time all_cases = pd.DataFrame([case_data(c) for c in eviction_cases], columns = columns)
all_cases.describe()

https://www.oscn.net/dockets/GetCaseInformation.aspx?db=tulsa&number=SC-2020-5645
https://www.oscn.net/dockets/GetCaseInformation.aspx?db=tulsa&number=SC-2020-5624
CPU times: user 490 ms, sys: 8.47 ms, total: 498 ms
Wall time: 2.26 s


,Filed,County,Type,Judge,Case Number,Source
count,2,2,2,2,2,2
unique,1,1,1,1,2,2
top,06/01/2020,tulsa,SC,"JUDGE, SMALL CLAIMS",tulsa-SC-2020-5624,https://www.oscn.net/dockets/GetCaseInformatio...
freq,2,2,2,2,1,1


In [27]:
all_cases

,Filed,County,Type,Judge,Case Number,Source
0,06/01/2020,tulsa,SC,"JUDGE, SMALL CLAIMS",tulsa-SC-2020-5645,https://www.oscn.net/dockets/GetCaseInformatio...
1,06/01/2020,tulsa,SC,"JUDGE, SMALL CLAIMS",tulsa-SC-2020-5624,https://www.oscn.net/dockets/GetCaseInformatio...
